# Recalibration of SMP density coefficients
*Josh King, Environment and Climate Change Canada, 2019*

The matched snow pit and SMP measurements from Part 1 are used to recalibrate the bilinear regression model of Proksch et al., (2015).


In [1]:
# Import community packages
import os
import string
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"

import matplotlib.cm as cm
import pickle

import scipy.stats
from scipy import stats
from statsmodels.formula.api import ols
from sklearn.model_selection import KFold, StratifiedShuffleSplit

# Seed to replicate the paper result exactly
RANDOM_SEED = 2019

# Load TVC comparison results from Part 1 
march = pd.read_pickle("./output/TVC/TVC_March2019_smp_pit_filtered_4.pkl") #March 2019
jan = pd.read_pickle("./output/TVC/TVC_Jan2019_smp_pit_filtered_incN_3.pkl") # Jan 2019 (Inc New Snow)

# Combine both campaigns
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
frames = [jan, march]
result = pd.concat(frames)

result['force_log'] = np.log(result['force_median'])

In [2]:
print (result)

    count_samp   mean_samp  median_samp  stdev_samp  index  bottom     id  \
0            7   22.598556    21.468414    4.066480      0    47.0  RP_01   
1            7  398.596814   409.300656   19.569935      1    44.0  RP_01   
2            7  225.770318   218.796716   39.438038      2    41.0  RP_01   
3            7  568.917286   571.620733   10.750147      3    38.0  RP_01   
4            7  542.346539   543.534441    3.679610      4    35.0  RP_01   
..         ...         ...          ...         ...    ...     ...    ...   
44           7  465.832246   488.325714   53.536323      3    27.0  SM_03   
45           7  309.951142   310.480971    3.153826      4    24.0  SM_03   
47           7  371.702372   363.129251   42.921041      6    18.0  SM_03   
48           7  294.654247   282.434343   27.079440      7    15.0  SM_03   
49           7  361.120992   356.268174   26.862623      8    12.0  SM_03   

      RHO   top TYPE  relative_height_mm         l  force_median       erro

In [22]:
#save to csv to use in matlab

result_df = pd.DataFrame(result)
print (result_df)
result_df.to_csv('./output/TVC/TVC_Merged_Comparison_4.csv',na_rep='NaN')

      RHO TYPE  bottom  count_samp       error  force_median     id  index  \
0   242.0    N    47.0           7 -219.401444      0.010784  RP_01      0   
1   221.0    N    44.0           7  177.596814      0.955946  RP_01      1   
2   153.0    N    41.0           7   72.770318      0.457522  RP_01      2   
3   371.0    F    38.0           7  197.917286      7.867569  RP_01      3   
4   385.0    F    35.0           7  157.346539      6.479529  RP_01      4   
..    ...  ...     ...         ...         ...           ...    ...    ...   
44  321.0    R    27.0           7  144.832246      4.866299  SM_03      3   
45  196.0    D    24.0           7  113.951142      0.126730  SM_03      4   
47  227.0    D    18.0           7  144.702372      0.092722  SM_03      6   
48  189.0    D    15.0           7  105.654247      0.215742  SM_03      7   
49  222.0    D    12.0           7  139.120992      0.084613  SM_03      8   

           l   mean_samp  median_samp  relative_height_mm  stde

In [5]:
#Compare manual density cutter measurements and SMP-derived densities
# P2015 evaluation stats
p2015_rmse = np.sqrt(np.mean(result['error']**2))
p2015_bias = (result['error']).mean()
p2015_r2 = np.ma.corrcoef(result['mean_samp'],result['RHO'])[0, 1]**2
p2015_n = len(result['mean_samp'])
p2015_p = stats.pearsonr(result['mean_samp'],result['RHO'])[1]

print('Proksch et al. 2015 Eval.')
print('N: %i' % p2015_n)
print('RMSE: %0.1f' % np.round(p2015_rmse))
print('bias: %0.1f' % np.round(p2015_bias))
print('r^2: %0.2f' % p2015_r2)


Proksch et al. 2015 Eval.
N: 228
RMSE: 125.0
bias: 95.0
r^2: 0.73


King 2020 Eval.
N: 228
RMSE: 125.0
bias: 95.0
r^2: 0.73


## K-Folds OLS method

OLS regression with 10 folds to minimize sampling bias.
Model coefficients and skill are evaluated as the mean of all folds.  

In [23]:
k_fold = KFold(10, True, RANDOM_SEED)
rmse = []
error = []
r = []
params = None

# Split the dataset into 10 roughly equal groups, 
# train on all but one test group
for train_idx, test_idx in k_fold.split(result):
    train = result.iloc[train_idx]
    test = result.iloc[test_idx]
    
    model_rho = ols("RHO ~ force_log + force_log * l + l", train).fit()
    predict_rho = model_rho.predict(exog=dict(force_log=test['force_log'], l=test['l']))
    rmse = np.append(rmse, np.sqrt(np.mean((predict_rho - test['RHO'])**2)))
    r = np.append(r,np.corrcoef(predict_rho, test['RHO'])[1][0])
    error = np.append(error, predict_rho - test['RHO'])
    
    if params is None:
        params = model_rho.params.values
    else:
        params =  np.vstack((params, model_rho.params.values))

In [24]:
# K19a evaluation stats (from k-folds method)
# Metrics presented as mean of all fold permutations

k19a_rmse = rmse.mean()
k19a_bias = error.mean()
k19a_r2 = r.mean()**2

print('K19a recalibration evaluation')
print('N: %i' % len(result))
print('RMSE: %0.1f' % k19a_rmse)
print('RMSE percent: %0.2f' % np.round(k19a_rmse/result.RHO.mean(),2))
print('bias: %0.1f' % k19a_bias)
print('r^2: %0.2f' % k19a_r2)

K19a recalibration evaluation
N: 228
RMSE: 35.1
RMSE percent: 0.13
bias: -0.0
r^2: 0.76


In [25]:
model_k19a_coeff = [np.round(params[:,0].mean(),2), np.round(params[:,1].mean(),2),
              np.round(params[:,3].mean(),2), np.round(params[:,2].mean(),2)]
var_coeffs = [np.round(params[:,0].std(),2), np.round(params[:,1].std(),2),
              np.round(params[:,3].std(),2), np.round(params[:,2].std(),2)]

# Unbiased coeffs
print(model_k19a_coeff)

# Save coeffs
filename = './output/TVC/TVC_Merged_density_k19a_coeffs_incN_4.pkl'
pickle.dump(model_k19a_coeff, open(filename, 'wb'))


[303.09, 38.89, -28.2, -61.15]


In [26]:
# Apply the new coeffs to estimate density
k19a_rho = model_k19a_coeff[0]+(model_k19a_coeff[1]*result['force_log'])+ \
           (model_k19a_coeff[2]*result['force_log']*result['l'])+ \
           model_k19a_coeff[3]*result['l']


In [27]:
#save new densities

k19a_rho_df = pd.DataFrame(k19a_rho)
print (k19a_rho_df)
k19a_rho_df.to_csv('./output/TVC/TVC_Merged_k19aDensities_incN_4.csv',na_rep='NaN')

             0
0   138.597049
1   295.512020
2   265.265236
3   365.407469
4   355.854250
..         ...
44  329.624086
45  219.106652
47  220.122352
48  220.617811
49  219.535478

[228 rows x 1 columns]


## K-folds OLS with outliers removed

Outliers were defined as SMP/Cutter comparisons where error > than the 95th quantile in the K19a recalibration.  
We justify this in the paper in the context of the matching procedure which cannot be assumed to be perfect.  
Note that this removes a small number of comparisons which are not isolated to any one profile.

In [28]:
#Remove outliers
result_lim = result.copy()
result_lim['f_l'] = (result_lim['l'])*result_lim['force_log']
result_lim['abs_error'] = np.abs(k19a_rho - result_lim['RHO'])
q_95 = result_lim['abs_error'].quantile(0.95) # This error threashold is still quite large, do I want to change this?
result_lim = result_lim[result_lim['abs_error'] < q_95]
n_removed  = len(result) - len(result_lim)

print('Error threshold: %i kg m^-3' % q_95 )
print('Data points removed: %i' % n_removed)

Error threshold: 67 kg m^-3
Data points removed: 12


In [29]:
q_95

67.80806272272523

#### Figure 5 with caption
![png](./output/figures/Fig05_RegressionTerms_lowres.png)
##### Comparison of the SMP regression parameters and corresponding snow density observations. Parameters include log-transformed median force (ln(F ̃)), microstructure length scale (L) and an interaction term (f ̃L). Relationships are separated by ice surface environment.'

In [33]:
# Correlation between snow pit observed density and median force.L for all comparisons
print(np.round(np.corrcoef(result_lim.RHO, result_lim.force_log)[0][1],2))

0.88


In [34]:
r, p = scipy.stats.pearsonr(result_lim.RHO, result_lim.force_log)
print('R2 = ' + str(r))
print('p = ' + str(p))

R2 = 0.8767822554224868
p = 5.624134292284972e-70


In [35]:
# Correlation between snow pit observed density and median force, microstructural length scale and the interaction term
result_lim[['RHO','force_log','l','f_l']].corr()

,RHO,force_log,l,f_l
RHO,1.000000,0.876782,-0.685092,0.667183
force_log,0.876782,1.000000,-0.637046,0.851955
l,-0.685092,-0.637046,1.000000,-0.764574
f_l,0.667183,0.851955,-0.764574,1.000000


In [36]:
k_fold = KFold(10, True, RANDOM_SEED)
rmse = []
error = []
r = []
params = None

for train_idx, test_idx in k_fold.split(result_lim):
    train = result_lim.iloc[train_idx]
    test = result_lim.iloc[test_idx]
    
    model_rho = ols("RHO ~ force_log + force_log * l + l", train).fit()
    predict_rho = model_rho.predict(exog=dict(force_log=test['force_log'], l=test['l']))
    rmse = np.append(rmse, np.sqrt(np.mean((predict_rho - test['RHO'])**2)))
    r = np.append(r,np.corrcoef(predict_rho, test['RHO'])[1][0])
    error = np.append(error, predict_rho - test['RHO'])
    
    if params is None:
        params = model_rho.params.values
    else:
        params =  np.vstack((params, model_rho.params.values))

In [37]:
# K19b evaluation stats (from kfolds method)
k19b_rmse = rmse.mean()
k19b_bias = error.mean()
k19b_r2 = r.mean()**2

print('K19b recalibration evaluation')
print('N: %i' % len(result_lim))
print('RMSE: %0.1f' % k19b_rmse)
print('RMSE percent: %0.2f' % np.round(k19b_rmse/result_lim.RHO.mean(),2))
print('bias: %0.1f' % k19b_bias)
print('r^2: %0.2f' % k19b_r2)

K19b recalibration evaluation
N: 216
RMSE: 25.2
RMSE percent: 0.09
bias: -0.2
r^2: 0.88


In [38]:
model_k19b_coeff = [np.round(params[:,0].mean(),2), np.round(params[:,1].mean(),2),
              np.round(params[:,3].mean(),2), np.round(params[:,2].mean(),2)]
var_coeffs = [np.round(params[:,0].std(),2), np.round(params[:,1].std(),2),
              np.round(params[:,3].std(),2), np.round(params[:,2].std(),2)]

# Unbiased coeffs
print(model_k19b_coeff)

# Save coeffs
filename = './output/TVC/TVC_Merged_density_k19b_coeffs_incN_4.pkl'
pickle.dump(model_k19b_coeff, open(filename, 'wb'))

[307.36, 43.51, -38.95, -79.36]


In [39]:
k19b_rho = model_k19b_coeff[0]+(model_k19b_coeff[1]*result_lim['force_log'])+ \
           (model_k19b_coeff[2]*result_lim['force_log']*result_lim['l'])+ \
           model_k19b_coeff[3]*result_lim['l']

#save new densities

k19b_rho_df = pd.DataFrame(k19b_rho)
print (k19b_rho_df)
k19b_rho_df.to_csv('./output/TVC/TVC_Merged_k19bDensities_incN_4.csv',na_rep='NaN')

             0
3   373.148024
4   362.060496
5   352.533566
6   245.196767
7   230.506700
..         ...
44  329.549149
45  218.865264
47  225.243724
48  215.604942
49  224.657474

[216 rows x 1 columns]


In [40]:
# Error metrics
def rmse(data):
    return np.sqrt(np.mean(data**2))

result_lim['model_rho'] = k19b_rho
result_lim['abs_error'] = np.abs(k19b_rho-result_lim['RHO']).values
result_lim['error'] = (k19b_rho-result_lim['RHO']).values

# Error by layer type
np.round(result_lim.groupby('TYPE')['error'].apply(rmse)/result_lim.groupby('TYPE')['model_rho'].mean(),3)

TYPE
D    0.095
F    0.089
N    0.084
R    0.089
dtype: float64

In [41]:
#RMSE by layer type
np.round(result_lim.groupby('TYPE')['error'].apply(rmse), 2)

TYPE
D    21.52
F    27.03
N    28.34
R    28.92
Name: error, dtype: float64

In [42]:
# Overall error in %
np.round((rmse(result_lim['error'])/result_lim['model_rho'].mean()),3)

0.091

In [44]:
#R^2 by layer type > Error is saying r2 is not defined, but it is
np.round(result_lim.groupby('TYPE')['error'].apply(k19b_r2), 2)

TypeError: 'numpy.float64' object is not callable